In [1]:
#import mvpa2
import pandas as pd
import numpy as np
#import sklearn
#to manipulate data
from mvpa2.datasets import *
import scipy.io

from mvpa2.suite import *
if __debug__:
    debug.active.append('SMLR_')
    
import matplotlib as plt
%matplotlib inline

//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/base/hdf5.py:40: H5pyDeprecationWarning: The h5py.highlevel module is deprecated, code should import directly from h5py, e.g. 'from h5py import File'.
  import h5py.highlevel  # >= 2.8.0, https://github.com/h5py/h5py/issues/1063
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/pydicom/__init__.py:55: DeprecationWarning: Python 2 will no longer be supported after the pydicom v1.4 release
  warnings.warn(msg, DeprecationWarning)
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/testing/tools.py:81: DeprecationWarning: Importing from numpy.testing.decorators is deprecated, import from numpy.testing instead.
  from numpy.testing.decorators import skipif


In [2]:
#makes a dict for task
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mem/MSC02_parcel_corrmat.mat')
#convert to numpy array
task_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
task_mask=np.triu_indices(nrois, 1)
task_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=task_FC[:,:,sess]
    task_corrlin[count]=tmp[task_mask]
    count=count+1

#makes a dict for rest
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/MSC02_parcel_corrmat.mat')
#convert to numpy array
rest_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
rest_mask=np.triu_indices(nrois, 1)
rest_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=rest_FC[:,:,sess]
    rest_corrlin[count]=tmp[rest_mask]
    count=count+1
    
#makes a dict for test task
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/motor/MSC02_parcel_corrmat.mat')
#convert to numpy array
motor_FC=np.array(matFile['parcel_corrmat'])
#Index of upper triangle of matrix 
nrois=333
nsess=10
test_mask=np.triu_indices(nrois, 1)
motor_corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=motor_FC[:,:,sess]
    motor_corrlin[count]=tmp[test_mask]
    count=count+1
#corrlin.shape
#task_ds=Dataset(corrlin)
#ds.shape
#ds.nfeatures
#ds.nsamples
#create a training dataset from the labeled features, targets 1:0 task:rest chunks=number of days
task_ds=dataset_wizard(samples=task_corrlin, targets=1, chunks=range(1,11))
rest_ds=dataset_wizard(samples=rest_corrlin, targets=0, chunks=range(1,11))
trainpat=vstack((task_ds, rest_ds))

#create pattern for testing dataset
test_ds=dataset_wizard(samples=motor_corrlin, targets=1)

In [53]:
#setup SMLR classifier
print "evaluating SMLR classifier"
smlr=SMLR(fit_all_weights=True)
#enable saving of estimates used for prediction
smlr.ca.enable('estimates')
#train with the known points
smlr.train(trainpat)
#run predictions on test values
pre=smlr.predict(test_ds.samples)
#caluclate confusion matrix
smlr_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets, predictions=pre)

evaluating SMLR classifier
[SMLR_] DBG:                     hstacking 1s for bias
[SMLR_] DBG:                     Calling stepwise_regression. Seed 275587169


In [61]:
#linear SVM
print "evaluating linear SVM classifier"
lsvm=LinearNuSVMC(probability=1)
#save estimates
lsvm.ca.enable('estimates')
#train with known points
lsvm.train(trainpat)
#run predictions on test values
pre=lsvm.predict(test_ds.samples)
#calculate confusion matrix
lsvm_confusion=ConfusionMatrix(labels=trainpat.UT, targets=test_ds.targets,predictions=pre)

evaluating linear SVM classifier


In [65]:
#train SVM with selected features
print "evaluating linear SVM classifier with SMLR features"
keepInd=(np.abs(smlr.weights).mean(axis=1)!=0)
newtrainpat=trainpat[:, keepInd]
newtestpat=test_ds[:, keepInd]
#train with known points
lsvm.train(newtrainpat)
#run prediction on test values
pre=lsvm.predict(newtestpat.samples)
#calculate confusion matrix
lsvm_confusion_sparse=ConfusionMatrix(labels=newtrainpat.UT, targets=newtestpat.targets, predictions=pre)


evaluating linear SVM classifier with SMLR features


In [50]:
print "SMLR Percent Correct:\t%g%% (Retained %d/%d features)" % \
    (smlr_confusion.percent_correct,
     (smlr.weights!=0).sum(), np.prod(smlr.weights.shape))
print "linear-SVM Percent Correct:\t%g%%" % \
    (lsvm_confusion.percent_correct)
print "linear-SVM Percent Correct (with %d features from SMLR):\t%g%%" % \
    (keepInd.sum(), lsvm_confusion_sparse.percent_correct)

SMLR Percent Correct:	60% (Retained 36/110556 features)
linear-SVM Percent Correct:	30%
linear-SVM Percent Correct (with 18 features from SMLR):	60%


In [58]:
smlr_confusion.matrix

array([[0, 4],
       [0, 6]])

In [14]:
trainpat.summary()

'Dataset: 20x55278@float64, <sa: targets> stats: mean=0.0280829 std=0.229266 var=0.052563 min=-1.05039 max=2.03915\nSequence statistics for 20 entries from set [0, 1]\nCounter-balance table for orders up to 2:\nTargets/Order O1    |  O2    |\n      0:       9 0  |   8 0  |\n      1:       1 9  |   2 8  |\nCorrelations: min=-1 max=0.8 mean=-0.053 sum(abs)=9'

In [93]:
#create objects holding linear svm, cross validation scheme==leave one out cross validation, accuracy
cv = CrossValidation(LinearCSVMC(), NFoldPartitioner(), 
                     enable_ca=['stats'], 
                     errorfx=lambda p, t: np.mean(p == t))

In [94]:
res = cv(trainpat)

In [95]:
res.summary()

'Dataset: 10x1@float64, <sa: cvfolds> stats: mean=0.95 std=0.15 var=0.0225 min=0.5 max=1\n'

In [96]:
np.mean(res.samples) - (1.0 / len(trainpat.UT)) # % above chance accuracy


0.44999999999999996

In [97]:
#CrossValidation?

In [101]:
np.mean(res.samples)


TypeError: unsupported operand type(s) for /: 'function' and 'int'

In [99]:
len(res)

10

In [100]:
res

Dataset(array([[1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [0.5]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='cvfolds', doc=None, value=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), length=10)]), fa=FeatureAttributesCollection(items=[]), a=DatasetAttributesCollection(items=[]))

In [92]:
cv.ca.stats.matrix

array([[10,  1],
       [ 0,  9]])

In [76]:
trainpat.shape

(20, 55278)

In [77]:
trainpat.summary

<bound method Dataset.summary of Dataset(array([[ 0.00095655, -0.07663061,  0.23755615, ...,  0.51705891,
         0.1983052 ,  0.03582447],
       [-0.20647831, -0.04299903,  0.43117783, ...,  0.31590771,
         0.26129547,  0.2377527 ],
       [-0.01635137, -0.11918725,  0.31998732, ...,  0.61881407,
         0.51909949,  0.27669716],
       ...,
       [-0.16529865, -0.08329263,  0.37580496, ...,  0.36276519,
         0.3779244 ,  0.29793391],
       [-0.07441239, -0.07626242,  0.31809336, ...,  0.36081897,
         0.47208516,  0.29183056],
       [-0.3631561 , -0.0513033 ,  0.16918412, ...,  0.36356048,
         0.51770127,  0.26640822]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='chunks', doc=None, value=array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10]), length=20), ArrayCollectable(name='targets', doc=None, value=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), length=20)]), fa=FeatureAttributes